In [6]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import gradio as gr
import pandas as pd
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))



In [7]:
# Các hàm của bạn
def flatten_image(img_path):
    img = cv2.imread(img_path)
    try:
        img = cv2.resize(img, (224, 224))
    except:
        print(img_path)
        return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img_array = np.expand_dims(img, axis=0)  # Thêm 1 trục (batch size)
    img_array = preprocess_input(img_array)
    return model.predict(img_array).flatten()

def flatten_list_dataset(root):
    root = os.path.join(root, 'images')
    feature_dict = {}
    role = ['tivi', 'smart_phone']
    for i, item in enumerate(role):
        imgs = os.listdir(os.path.join(root, item))
        for img_path in imgs:
            features = flatten_image(os.path.join(root, item, img_path))
            if features is not None:
                feature_dict[os.path.join(root, item, img_path)] = features.flatten()
    return feature_dict


dataset = flatten_list_dataset('dataset')



1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━

In [8]:
def pred_by_cosine_similarity(pred_img_path, data):
    features = flatten_image(pred_img_path)

    feature_vectors = list(data.values())
    labels = list(data.keys())

    similarities = cosine_similarity([features], feature_vectors)[0]

    top_indices = similarities.argsort()[-5:][::-1]

    top_keys = [labels[i] for i in top_indices]
    top_similarities = [similarities[i] for i in top_indices]
    for i, _ in enumerate(top_similarities):
        top_similarities[i] = (1 - top_similarities[i]) * 100

    result_images = []
    for i in range(5):
        similar_img = cv2.imread(top_keys[i])
        similar_img = cv2.cvtColor(cv2.resize(similar_img, (224, 224)), cv2.COLOR_BGR2RGB)
        result_images.append((similar_img, f"Similarity: {top_similarities[i]:.2f}"))

    for i in range(5):
        top_keys[i] = os.path.basename(top_keys[i])

    return [result_images, top_keys, top_similarities]  


In [9]:
sample_dir = 'test_data'
sample_paths = [os.path.join(sample_dir, img) for img in os.listdir(sample_dir)]

# Hàm tìm hình ảnh tương tự
def find_similar_images(img):
    # Lấy kết quả hình ảnh và tên
    result_images = pred_by_cosine_similarity(img, dataset)[0]
    result_names = pred_by_cosine_similarity(img, dataset)[1]
    
    return result_images, result_names

# Hàm tìm ID và tên sản phẩm từ DataFrame
def find_product_info(result_names):
    # Đọc file Excel
    df = pd.read_excel("dataset/output.xlsx")
    list_id, list_product_name = [], []

    for name in result_names:
        list_id.append(df[df['name'] == name]['id'].values[0])  # Lấy giá trị đầu tiên
        list_product_name.append(df[df['name'] == name]['product name'].values[0])  # Lấy giá trị đầu tiên

    return list_id, list_product_name

# Hàm xử lý giao diện Gradio
def gradio_interface(img):
    # Tìm hình ảnh tương tự
    result_images, result_names, result_similarities = pred_by_cosine_similarity(img, dataset)

    # Tìm ID và tên sản phẩm
    list_id, list_product_name = find_product_info(result_names)

    # Tạo output với ảnh, id, tên sản phẩm và khoảng cách cosine
    output = []
    for i in range(5):
        img_with_info = {
            "image": result_images[i][0],
            "label": f"ID: {list_id[i]}, Product: {list_product_name[i]}, Cos_dis: {result_similarities[i]:.2f}%"
        }
        output.append((img_with_info["image"], img_with_info["label"]))

    return output


# Tạo giao diện Gradio với chức năng add_examples
with gr.Blocks() as demo:
    gr.Markdown("Upload an image to find the top 5 similar images using cosine similarity or select a sample image.")

    input_image = gr.Image(type="filepath", label="Upload an Image")

    def handle_input(uploaded_img=None):
        if uploaded_img:  # Sử dụng ảnh người dùng upload hoặc example
            return gradio_interface(uploaded_img)
        return []

    output_gallery = gr.Gallery(label="Top 5 Similar Images", columns=5, height="auto")
    gr.Examples(examples=sample_paths, inputs=[input_image], outputs=output_gallery, fn=handle_input)

    # Nút bấm cho ảnh người dùng upload
    submit_btn = gr.Button("Find Similar Images")
    submit_btn.click(fn=handle_input, inputs=[input_image], outputs=output_gallery)

# Chạy giao diện Gradio
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://a055ee50707ffc7db7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
